# Toxic comments pre-processing using spacy

In [114]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [18]:
df1 = pd.read_csv('train.csv')
df1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


### Load the statistical model of spacy

In [20]:
nlp = spacy.load('en_core_web_sm',disable=["tagger", "parser"])

# Converts "wasn't" to "was","n't" 
# A new column with comment_text tokenized
df1['tokenized_sents'] = df1.apply(lambda row: nlp(row['comment_text']), axis=1) 


In [135]:
word2seq = {}
seq2word = {}
num = 1

df1['Converted'] = ''

for i in range(len(df1)):
    df1['Converted'][i] = []
    for token in df1['tokenized_sents'][i]:
        # To filter out STOP words - more frequently occuring words
        if token.is_stop == False: 
            # Lowercase all alphabets
            low_txt = token.text.lower() 
            if low_txt not in word2seq: 
                word2seq[low_txt] = num
                seq2word[num] = low_txt
                num = num + 1
            df1['Converted'][i].append(int(word2seq[low_txt]))

E:\SiddC\ML_ANN\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [136]:
type(np.stack(df1['Converted'][0]))

numpy.ndarray

### Cross verifying the tokenized sentence

In [137]:
sent_numb = 20

print(' length of dictionary :', len(word2seq), df1['comment_text'][sent_numb])
print(' length of tokenized list :', len(df1['tokenized_sents'][sent_numb]),'\n', 'length of converted list :', len(df1['Converted'][sent_numb]))

 length of dictionary : 219926 "

 Regarding your recent edits 

Once again, please read WP:FILMPLOT before editing any more film articles.  Your edits are simply not good, with entirely too many unnecessary details and very bad writing.  Please stop before you do further damage. -''''''The '45 "
 length of tokenized list : 54 
 length of converted list : 33


There are 54 words in total. Among them it looks like there are 21 STOP(common) words. Lets check what are the STOP words in that particular sentence.

In [138]:
i = 1
for ind in df1['Converted'][sent_numb]:
    print(i,'-->',seq2word[ind])
    i = i+1

1 --> "
2 --> 

 
3 --> recent
4 --> edits
5 --> 


6 --> ,
7 --> read
8 --> wp
9 --> :
10 --> filmplot
11 --> editing
12 --> film
13 --> articles
14 --> .
15 -->  
16 --> edits
17 --> simply
18 --> good
19 --> ,
20 --> entirely
21 --> unnecessary
22 --> details
23 --> bad
24 --> writing
25 --> .
26 -->  
27 --> stop
28 --> damage
29 --> .
30 --> -''''''the
31 --> '
32 --> 45
33 --> "


### Save the dictionary and dataframe for training.

In [139]:
# Save word2seq dictionary
np.save('word2seq.npy', word2seq) 

#Save seq2word dictionary
np.save('seq2word.npy', seq2word) 

# Save Dataframe
df1.to_csv('modified_train.csv',index = False)